TOTO予測
===

# データ収集

[公式記録](https://data.j-league.or.jp/)からデータを収集する。

APIとかで取得できるようにはなっていないので、HTMLをパースして加工しやすいような形式にすることを考える。

`https://data.j-league.or.jp/SFMS01/search?competition_years=<年度>&competition_frame_ids=<カテゴリ>`

In [1]:
import requests
from bs4 import BeautifulSoup

In [3]:
year = 2018
competition = 1

url = 'https://data.j-league.or.jp/SFMS01/search?competition_years='+ str(year) + '&competition_frame_ids=' + str(competition)

print("URL= " + url)
resp = requests.get(url)
#print(resp.text)

URL= https://data.j-league.or.jp/SFMS01/search?competition_years=2018&competition_frame_ids=1


HTMLを直接見ると、tableタグのデータを見れば良さそう。

Tableタグの取得は、pandasで簡単に取れるらしい([参考](https://qiita.com/kitsuyui/items/4906bb457af4d0e2d0a5))ので、これを使う。

In [4]:
import pandas as pd
from IPython.core.display import display
dfs = pd.io.html.read_html(url)
dfs[0] = dfs[0].drop(['インターネット中継・TV放送','入場者数'],axis=1) # 結果に関係なさそうな放送局と入場者数はこの時点で削除しておく
display(dfs[0])

,年度,大会,節,試合日,K/O時刻,ホーム,スコア,アウェイ,スタジアム
0,2018,Ｊ１,第１節第１日,02/23(金),20:03,鳥栖,1-1,神戸,ベアスタ
1,2018,Ｊ１,第１節第２日,02/24(土),14:03,FC東京,1-1,浦和,味スタ
2,2018,Ｊ１,第１節第２日,02/24(土),14:03,広島,1-0,札幌,Ｅスタ
3,2018,Ｊ１,第１節第２日,02/24(土),14:33,Ｇ大阪,2-3,名古屋,吹田Ｓ
4,2018,Ｊ１,第１節第２日,02/24(土),16:03,湘南,2-1,長崎,ＢＭＷス
5,2018,Ｊ１,第１節第３日,02/25(日),13:02,仙台,1-0,柏,ユアスタ
6,2018,Ｊ１,第１節第３日,02/25(日),13:03,清水,0-0,鹿島,アイスタ
7,2018,Ｊ１,第１節第３日,02/25(日),15:31,磐田,0-3,川崎Ｆ,エコパ
8,2018,Ｊ１,第１節第３日,02/25(日),16:03,Ｃ大阪,1-1,横浜FM,ヤンマー
9,2018,Ｊ１,第２節第１日,03/02(金),19:03,川崎Ｆ,1-1,湘南,等々力


ここまでで、[download.py](download.py)の関数download_resultを作成したので、これで試してみる。

In [5]:
import download
df = download.download_result(year=1992)
display(df)

,年度,大会,節,試合日,K/O時刻,ホーム,スコア,アウェイ,スタジアム,入場者数,インターネット中継・TV放送
0,1992,ＹＮＣ,第１節第１日,09/05(土),15:00,鹿島,4-2,横浜Ｆ,笠松,5226,NaN
1,1992,ＹＮＣ,第１節第１日,09/05(土),15:30,名古屋,3-0,清水,瑞穂球,8029,NaN
2,1992,ＹＮＣ,第１節第１日,09/05(土),18:02,浦和,2-3,市原,大宮,4934,NaN
3,1992,ＹＮＣ,第１節第１日,09/05(土),19:00,Ｇ大阪,2-0,横浜M,神戸中央,4728,NaN
4,1992,ＹＮＣ,第１節第２日,09/06(日),19:00,広島,2-3,Ｖ川崎,広島ス,13861,NaN
5,1992,ＹＮＣ,第２節第１日,09/09(水),18:30,広島,2-3,浦和,広島ス,3128,NaN
6,1992,ＹＮＣ,第２節第１日,09/09(水),19:00,鹿島,4-3,Ｖ川崎,国立,18009,NaN
7,1992,ＹＮＣ,第２節第１日,09/09(水),19:00,清水,2-1,横浜M,日本平,8032,NaN
8,1992,ＹＮＣ,第２節第１日,09/09(水),19:00,名古屋,2-1,横浜Ｆ,瑞穂球,5165,NaN
9,1992,ＹＮＣ,第２節第１日,09/09(水),19:00,Ｇ大阪,1-0,市原,神戸中央,2760,NaN


CSVとかでいいけど、最近データベースを勉強中なのでsqlite3に保存してみる  
pandasのDataFrameはto_sqlメソッドでSQLに変換可能らしい([参考](http://www.stockdog.work/entry/2017/03/22/010057))。

保存場所として、data/jleague.dbとするものとした。  
この場合、dataフォルダが存在しない場合は作ることにする。

In [6]:
import sqlite3
import os
db_name = "data/test.db"
if os.path.exists('data') == False:
    os.mkdir('data')
conn = sqlite3.connect(db_name) # DB接続
dfs[0].to_sql("result", conn, if_exists='replace') # DFをDBに変換

ここまでで、download_all_resultsにした。

download_all_resultsは、1992年～2017年のデータを取得し、sqlite(data/jleague.db)に変換する。  
jleague.dbのresultというデータベースに、取得した結果を格納する。

In [7]:
download.download_all_results()

生成されたDBを見てみる。

In [8]:
import sqlite3
db_name = 'data/jleague.db'
conn = sqlite3.connect(db_name)
df = pd.read_sql_query("select * from result", conn)
display(df)
conn.close()

,index,年度,大会,節,試合日,K/O時刻,ホーム,スコア,アウェイ,スタジアム,入場者数,インターネット中継・TV放送
0,0,1992,ＹＮＣ,第１節第１日,09/05(土),15:00,鹿島,4-2,横浜Ｆ,笠松,5226.0,None
1,1,1992,ＹＮＣ,第１節第１日,09/05(土),15:30,名古屋,3-0,清水,瑞穂球,8029.0,None
2,2,1992,ＹＮＣ,第１節第１日,09/05(土),18:02,浦和,2-3,市原,大宮,4934.0,None
3,3,1992,ＹＮＣ,第１節第１日,09/05(土),19:00,Ｇ大阪,2-0,横浜M,神戸中央,4728.0,None
4,4,1992,ＹＮＣ,第１節第２日,09/06(日),19:00,広島,2-3,Ｖ川崎,広島ス,13861.0,None
5,5,1992,ＹＮＣ,第２節第１日,09/09(水),18:30,広島,2-3,浦和,広島ス,3128.0,None
6,6,1992,ＹＮＣ,第２節第１日,09/09(水),19:00,鹿島,4-3,Ｖ川崎,国立,18009.0,None
7,7,1992,ＹＮＣ,第２節第１日,09/09(水),19:00,清水,2-1,横浜M,日本平,8032.0,None
8,8,1992,ＹＮＣ,第２節第１日,09/09(水),19:00,名古屋,2-1,横浜Ｆ,瑞穂球,5165.0,None
9,9,1992,ＹＮＣ,第２節第１日,09/09(水),19:00,Ｇ大阪,1-0,市原,神戸中央,2760.0,None
